In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append("../../") 

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
import matplotlib.gridspec as gridspec
plt.rcParams["font.family"] = "Optima"
plt.rcParams["font.weight"] = "light"

import numpy as np
import arviz as az

import yaml

from epimodel import preprocess_data, run_model, EpidemiologicalParameters
from epimodel.plotting.intervention_plots import combine_npi_samples, plot_intervention_effectiveness, plot_intervention_correlation

In [ ]:
sns.color_palette('colorblind')

In [ ]:
cols = sns.color_palette('colorblind')

In [ ]:
data = preprocess_data('../../data/all_merged_data_2021-01-22.csv')
data.featurize() 
data.mask_new_variant(new_variant_fraction_fname='../../data/nuts3_new_variant_fraction.csv')
data.mask_from_date('2021-01-09') 

In [ ]:
cm_names = data.CMs

In [ ]:
full_trace = az.from_netcdf('final_results2.netcdf')
alpha_i = full_trace.posterior.alpha_i.data.reshape((5000, 19))

In [ ]:
100*(1-np.percentile(np.exp(-np.sum(alpha_i, axis=-1)), [97.5, 50, 2.5]))

In [ ]:
full_trace = az.from_netcdf('final_results2.netcdf')

In [ ]:
full_trace.posterior.r_walk_noise_scale.data.flatten()

In [ ]:
alpha_i = full_trace.posterior.alpha_i.data.reshape((5000, 19))

# Main Results Figure

In [ ]:
gatherings_dict = {
    'All public gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤6 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤6 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'All household mixing in private banned': {
        'npis': ['Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤6 people from 2 Households': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤6 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 6',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    }
}


In [ ]:
main_panel_dict = {
    'All non-essential\nbusinesses closed': {
        'npis': ['Retail Closed', 'Some Face-to-Face Businesses Closed', 
                'Gastronomy Closed', 'Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': True,
    },
    'Night clubs closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
        'main': False,
    },
    'Leisure and entertainment\nvenues closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Gastronomy closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Retail and close-contact\nservices closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': False,
    },
    'All gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': True
    },
    'All gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All educational\ninstitutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': True
    },
    'Night time curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'Stricter mask-wearing\npolicy': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[6],
        'main': True
    },
}

gatherings_dict = {
    'All public gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'All household mixing in private banned': {
        'npis': ['Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people from 2 households': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    }
}

In [ ]:
def create_panel(ax, npi_dict, alpha_i, cm_names):
    comb_cm_effects, new_names = combine_npi_samples(npi_dict, alpha_i, cm_names)

    npi_cols = [d['color'] for k, d in npi_dict.items()]
    per_red = 100 * (1 - np.exp(-comb_cm_effects))
    median_alpha = np.median(per_red, axis=0)
    nS, nCMs = per_red.shape

#     for i in range(0, nCMs, 2):
#         ax.fill_between(
#             [-100, 100],
#             [-i + 0.5, -i + 0.5],
#             [-i - 0.5, -i - 0.5],
#             color="tab:grey",
#             alpha=0.1,
#             linewidth=0,
#         )

    li, lq, uq, ui = np.percentile(per_red, [2.5, 25, 75, 97.5], axis=0)

    for n in range(nCMs):
        ax.plot([li[n], ui[n]], [-n, -n], color=npi_cols[n], alpha=0.35*0.35, linewidth=2, solid_capstyle='round')
        ax.plot([lq[n], uq[n]], [-n, -n], color=npi_cols[n], alpha=0.85*0.35, linewidth=2, solid_capstyle='round')

    ax.plot([0, 0], [0.5, -nCMs - 2], "k--", linewidth=0.5, zorder=-5, color="tab:gray")
    ax.set_yticks(ticks=-np.arange(len(median_alpha)))
    ax.set_yticklabels(new_names, fontdict={"fontsize": 10})

    for i, tick in enumerate(ax.get_yticklabels()):
        tick.set_color(npi_cols[i])

    ax.scatter(median_alpha, -np.arange(len(median_alpha)), marker="o", color=npi_cols, s=18, edgecolors = npi_cols, facecolor="white", zorder=5)
    ax.set_xlabel("Reduction in R (%)")
    ax.set_ylim([-nCMs + 0.5, 0.5])
    ax.set_xlim([-10, 50])

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(5.75, 8.25), dpi=400)
gs = gridspec.GridSpec(ncols=8, nrows=11, figure=fig)

main_ax = fig.add_subplot(gs[:8, :])

npi_dict = main_panel_dict
ax = main_ax

groups = [(0, 4), (5, 9), (10, 10), (11, 11) , (12, 12)]
g_cols = [cols[0], cols[3], cols[2], cols[1], cols[6]]
marker_vals = [0, 5, 10, 11, 12]

comb_cm_effects, new_names = combine_npi_samples(main_panel_dict, alpha_i, cm_names)
npi_cols = [d['color'] for k, d in npi_dict.items()]
is_mains = [d['main'] for k, d in npi_dict.items()]
per_red = 100 * (1 - np.exp(-comb_cm_effects))
median_alpha = np.median(per_red, axis=0)
nS, nCMs = per_red.shape

for g, col in zip(groups, g_cols):
    start = g[0]-1
    end = g[1]+1
    ax.fill_between(
        [-100, 100],
        [-end + 0.5, -end + 0.5],
        [-start - 0.5, -start - 0.5],
        color=col,
        alpha=0.04,
        linewidth=0,
    )
    
for end in marker_vals:
    plt.plot([-100, 100], [-end+0.5, -end+0.5], color='k', alpha=0.65)
    plt.plot([-100, 100], [-end-0.5, -end-0.5], color='k', alpha=0.65)
plt.plot([-100, 100], [-12-0.5, -12-0.5], color='k', alpha=0.65)


li, lq, uq, ui = np.percentile(per_red, [2.5, 25, 75, 97.5], axis=0)

for n in range(nCMs):
    if is_mains[n]:
        ax.plot([li[n], ui[n]], [-n, -n], color=npi_cols[n], alpha=0.35, linewidth=3, solid_capstyle='round')
        ax.plot([lq[n], uq[n]], [-n, -n], color=npi_cols[n], alpha=0.85, linewidth=3, solid_capstyle='round')
    else:
        ax.plot([li[n], ui[n]], [-n, -n], color=npi_cols[n], alpha=0.35*0.35, linewidth=2, solid_capstyle='round')
        ax.plot([lq[n], uq[n]], [-n, -n], color=npi_cols[n], alpha=0.85*0.35, linewidth=2, solid_capstyle='round')

ax.plot([0, 0], [0.5, -nCMs - 2], "k--", linewidth=0.5, zorder=-5, color="tab:gray")
plt.yticks(-np.arange(len(median_alpha)), new_names, linespacing=0.8)

for i, tick in enumerate(ax.get_yticklabels()):
    tick.set_color(npi_cols[i])
    if is_mains[i]:
        tick.set_size(12)
    else:
        tick.set_size(9)

ax.scatter(median_alpha, -np.arange(len(median_alpha)), marker="o", color=npi_cols, s=18, edgecolors = npi_cols, facecolor="white", zorder=5)
ax.set_xlabel("Reduction in R (%)")
ax.set_ylim([-nCMs + 0.5, 0.5])
ax.set_xlim([-10, 50])

gath_ax = fig.add_subplot(gs[8:, :])
gath_ax.fill_between(
        [-100, 100],
        [-4.5, -4.5],
        [0.5, 0.5],
        color=cols[6],
        alpha=0.04,
        linewidth=0,
    )

gath_ax.fill_between(
        [-100, 100],
        [-10.5, -10.5],
        [-4.5, -4.5],
        color=cols[5],
        alpha=0.04,
        linewidth=0,
    )

create_panel(gath_ax, gatherings_dict, alpha_i, cm_names)
plt.plot([-100, 100], [-4.5, -4.5], color='k', alpha=0.65)
plt.yticks(fontsize=7, linespacing=0.8)
plt.savefig('FigMain.pdf')
plt.savefig('FigMain.svg')

# Specific Combinations

In [ ]:
text_dict = {
    'All non-essential\nbusinesses closed': {
        'npis': ['Retail Closed', 'Some Face-to-Face Businesses Closed', 
                'Gastronomy Closed', 'Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': True,
    },
    'Night clubs closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
        'main': False,
    },
    'Leisure and entertainment\nvenues closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Gastronomy closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Retail and close-contact\nservices closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': False,
    },
    'lockdown': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit',
                 'Retail Closed', 
                 'Some Face-to-Face Businesses Closed', 
                 'Gastronomy Closed', 
                 'Leisure Venues Closed'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': True
    },
    'All gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': True
    },
    'All gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
        'main': False
    },
    'All educational\ninstitutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': True
    },
    'Night time curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'Stricter mask-wearing\npolicy': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[6],
        'main': True
    },
    'All public gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'All household mixing in private banned': {
        'npis': ['Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people from 2 households': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    }
}

In [ ]:
comb_cm_effects, new_names = combine_npi_samples(text_dict, alpha_i, cm_names)
per_red = 100 * (1 - np.exp(-comb_cm_effects))

In [ ]:
for cm_i, name in enumerate(new_names):
    print(f"{name}: {np.percentile(per_red[:, cm_i], [2.5, 50, 97.5])}")

# Effect of All NPIs Combined

In [ ]:
100*(1-np.percentile(np.exp(-np.sum(alpha_i, axis=-1)), [97.5, 50, 2.5]))

# Other Total Effect Metrics

In [ ]:
data.active_cms.shape

In [ ]:
# compute total reduction due to active cms at each point in time
total_alpha_i = (alpha_i.reshape((5000, 1, 19, 1)) * data.active_cms.reshape((1, 114, 19, 175))).sum(axis=2)

In [ ]:
total_effect = 100*(1-np.exp(-total_alpha_i))

In [ ]:
max_total_effect = np.max(total_effect, axis=-1)

In [ ]:
np.percentile(max_total_effect, [2.5, 50, 97.5])

In [ ]:
Rt = full_trace.posterior['Rt'].data.reshape((5000, 114, 175))

In [ ]:
# Rt at start
np.percentile(Rt[:, :, 0], [2.5, 50, 97.5])

In [ ]:
last_days = []
for r_i in range(data.nRs):
    ds = np.nonzero(data.new_cases.mask[r_i, :])[0]
    last_days.append(ds[8]-5)
last_days = np.array(last_days)

In [ ]:
# Rt at end
np.percentile(Rt[:, :, 0], [2.5, 50, 97.5])
last_days_Rt = np.zeros((5000, data.nRs))

for r_i in range(data.nRs):
    last_days_Rt[:, r_i] = Rt[:, r_i, last_days[r_i]]
    np.percentile(last_days_Rt, [2.5, 50, 97.5])

In [ ]:
np.percentile(np.min(full_trace.posterior.Rt_walk.data, axis=-1), [2.5, 50, 97.5])

In [ ]:
np.percentile(np.max(full_trace.posterior.Rt_walk.data, axis=-1), [2.5, 50, 97.5])

In [ ]:
# max Rt
np.percentile(np.max(Rt, axis=-1), [2.5, 50, 97.5])

In [ ]:
# min Rt
np.percentile(np.min(Rt, axis=-1), [2.5, 50, 97.5])

In [ ]:
# max to min reduction
# max Rt
np.percentile(np.max(Rt, axis=-1) - np.min(Rt, axis=-1) , [2.5, 50, 97.5])

In [ ]:
# maximum additive effect per region

In [ ]:
Rt_walk = full_trace.posterior.Rt_walk.data.reshape((5000, 114, 175))

In [ ]:
total_additive_reduction = Rt_walk*(1-np.exp(-total_alpha_i))

In [ ]:
total_additive_reduction.shape

In [ ]:
# additive instantiated
np.percentile(np.max(total_additive_reduction, axis=-1), [2.5, 50, 97.5])

In [ ]:
Rt_walk.shape

In [ ]:
np.percentile(np.max(Rt_walk *(1 - np.exp(-np.sum(alpha_i, axis=-1)).reshape((5000, 1, 1))), axis=-1), [2.5, 50, 97.5])

# Plots needed elsewhere in the manuscript

## MCMC Plot

In [ ]:
ess = az.ess(full_trace)

In [ ]:
rhat = az.rhat(full_trace)

In [ ]:
def collate(stat):
    stat_all = []
    stat_nums = []
    for var in ["Rt_walk", "Rt_cm", "Rt_walk", "alpha_i", "basic_R", "cfr", "expected_cases", "expected_deaths", "total_infections", 
                "future_cases_t", "future_deaths_t", "infection_noise", "psi_cases", "psi_deaths", "r_walk_noise", "r_walk_noise_scale", "seeding", "total_infections"]:
        if stat[str(var)].size>1:
            stat_all.append(stat[str(var)].to_dataframe().to_numpy().flatten())
        else:
            stat_nums.append(float(stat[str(var)]))
    stat_all = np.concatenate(np.array(stat_all))
    stat_all = np.concatenate([stat_all, stat_nums])
    return stat_all

In [ ]:
sns.set_palette('colorblind')

plt.figure(figsize=(6, 3), dpi=300)
plt.subplot(121)
sns.histplot(collate(rhat), element='bars', bins=100)
plt.xlabel("$\hat{R}$", fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.ylabel("Count", fontsize=10)

plt.subplot(122)
sns.histplot(collate(ess), element='bars', bins=100)
plt.xlabel("Effective Samples", fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.ylabel("Count", fontsize=10)

plt.tight_layout()
plt.savefig('figures/appendix/FigMCMC.pdf', bbox_inches='tight')

## Model Fits

In [ ]:
main_panel_dict = {
    'All non-essential\nbusinesses closed': {
        'npis': ['Retail Closed', 'Some Face-to-Face Businesses Closed', 
                'Gastronomy Closed', 'Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': True,
    },
    'Night clubs closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
        'main': False,
    },
    'Leisure and entertainment\nvenues closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Gastronomy closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Retail and close-contact\nservices closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': False,
    },
    'All educational\ninstitutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': True
    },
    'Schools closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'Universities closed': {
        'npis': ['Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'All public gatherings banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people\nfrom 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'All household mixing in private banned': {
        'npis': ['Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people from 2 households': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Night time curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'Stricter mask-wearing\npolicy': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
}

In [ ]:
from epimodel.plotting.epicurve_plots import plot_area_cases_curve, plot_area_transmission_curve, plot_area_infections_curve, plot_area_deaths_curve, add_cms_to_plot

In [ ]:
data.nDs

In [ ]:
for r_i in range(data.nRs):
    plt.figure(figsize=(8, 3), dpi=400)
    plt.subplot(331)
    area_Rt_samples = full_trace.posterior['Rt'][:, :, r_i, :].data.reshape((5000, 175))
    plot_area_transmission_curve(area_Rt_samples, data.Ds)

    plt.subplot(332)
    area_infections = full_trace.posterior["total_infections"].data[:, :, r_i, 7:].reshape((5000, 175))
    plot_area_infections_curve(area_infections, data.Ds)
    plt.title(data.Rs[r_i], fontsize=12)

    plt.subplot(333)
    expected_cases = full_trace.posterior["expected_cases"][:, :, r_i, :].data.reshape((5000, 175))
    psi_cases = full_trace.posterior["psi_cases"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
    new_cases = data.new_cases[r_i, :]

    expected_deaths = full_trace.posterior["expected_deaths"][:, :, r_i, :].data.reshape((5000, 175))
    psi_deaths = full_trace.posterior["psi_deaths"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
    new_deaths = data.new_deaths[r_i, :]

    plot_area_cases_curve(expected_cases, psi_cases, new_cases, data.Ds)
    plt.ylim([1, 10000])
    plt.ylabel("Cases", fontsize=10, color="tab:blue")

    plt.minorticks_off()
    plt.twinx()

    plot_area_deaths_curve(expected_deaths, psi_deaths, new_deaths, data.Ds)
    plt.ylabel("Deaths", fontsize=10, rotation=-90, color="tab:red", va='bottom')
    plt.ylim([1, 10000])
    plt.yticks([])
    plt.minorticks_off()
    plt.tight_layout()
    plt.show()

In [ ]:
for r_i in range(data.nRs):
    if r_i%36 == 0:
        plt.figure(figsize=(10, 10), dpi=400)
            
    plt.subplot(6,6, r_i%36 + 1)
    area_Rt_samples = full_trace.posterior['Rt_walk'][:, :, r_i, :].data.reshape((5000, 175))
    plot_area_transmission_curve(area_Rt_samples, data.Ds)
    plt.ylabel('Random Walk')
    plt.title(data.Rs[r_i], fontsize=8)
    plt.xticks(fontsize=4, ha='left', rotation=-30)
    
    if r_i%36 == 35 or r_i == data.nRs - 1:
        if r_i > 0:
            plt.tight_layout()
            plt.savefig(f'RW_resid_{r_i//36}.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(8, 6), dpi=400)

r_i = data.Rs.index(Rs[0])

# 1
plt.subplot(331)
area_Rt_samples = full_trace.posterior['Rt'][:, :, r_i, :].data.reshape((5000, 175))
plot_area_transmission_curve(area_Rt_samples, data.Ds)

plt.subplot(332)
area_infections = full_trace.posterior["total_infections"].data[:, :, r_i, 7:].reshape((5000, 175))
plot_area_infections_curve(area_infections, data.Ds)
plt.title(Rs[0], fontsize=12)

plt.subplot(333)
expected_cases = full_trace.posterior["expected_cases"][:, :, r_i, :].data.reshape((5000, 175))
psi_cases = full_trace.posterior["psi_cases"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
new_cases = data.new_cases[r_i, :]

expected_deaths = full_trace.posterior["expected_deaths"][:, :, r_i, :].data.reshape((5000, 175))
psi_deaths = full_trace.posterior["psi_deaths"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
new_deaths = data.new_deaths[r_i, :]

plot_area_cases_curve(expected_cases, psi_cases, new_cases, data.Ds)
plt.ylim([1, 10000])
plt.ylabel("Cases", fontsize=10, color="tab:blue")

plt.minorticks_off()
plt.twinx()

plot_area_deaths_curve(expected_deaths, psi_deaths, new_deaths, data.Ds)
plt.ylabel("Deaths", fontsize=10, rotation=-90, color="tab:red", va='bottom')
plt.ylim([1, 10000])
plt.yticks([])
plt.minorticks_off()

## 2

r_i = data.Rs.index(Rs[1])

# 1
plt.subplot(334)
area_Rt_samples = full_trace.posterior['Rt'][:, :, r_i, :].data.reshape((5000, 175))
plot_area_transmission_curve(area_Rt_samples, data.Ds)

plt.subplot(335)
area_infections = full_trace.posterior["total_infections"].data[:, :, r_i, 7:].reshape((5000, 175))
plot_area_infections_curve(area_infections, data.Ds)
plt.title(Rs[1], fontsize=12)

plt.subplot(336)
expected_cases = full_trace.posterior["expected_cases"][:, :, r_i, :].data.reshape((5000, 175))
psi_cases = full_trace.posterior["psi_cases"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
new_cases = data.new_cases[r_i, :]

expected_deaths = full_trace.posterior["expected_deaths"][:, :, r_i, :].data.reshape((5000, 175))
psi_deaths = full_trace.posterior["psi_deaths"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
new_deaths = data.new_deaths[r_i, :]

plot_area_cases_curve(expected_cases, psi_cases, new_cases, data.Ds)
plt.ylim([1, 10000])
plt.ylabel("Cases", fontsize=10, color="tab:blue")

plt.minorticks_off()
plt.twinx()

plot_area_deaths_curve(expected_deaths, psi_deaths, new_deaths, data.Ds)
plt.ylabel("Deaths", fontsize=10, rotation=-90, color="tab:red", va='bottom')
plt.ylim([1, 10000])
plt.yticks([])
plt.minorticks_off()

r_i = data.Rs.index(Rs[2])

# 3
plt.subplot(337)
area_Rt_samples = full_trace.posterior['Rt'][:, :, r_i, :].data.reshape((5000, 175))
plot_area_transmission_curve(area_Rt_samples, data.Ds)

plt.subplot(338)
area_infections = full_trace.posterior["total_infections"].data[:, :, r_i, 7:].reshape((5000, 175))
plot_area_infections_curve(area_infections, data.Ds)
plt.title(Rs[2], fontsize=12)

plt.subplot(339)
expected_cases = full_trace.posterior["expected_cases"][:, :, r_i, :].data.reshape((5000, 175))
psi_cases = full_trace.posterior["psi_cases"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
new_cases = data.new_cases[r_i, :]

expected_deaths = full_trace.posterior["expected_deaths"][:, :, r_i, :].data.reshape((5000, 175))
psi_deaths = full_trace.posterior["psi_deaths"][:, :, data.unique_Cs.index(data.Cs[r_i])].data.reshape((5000))
new_deaths = data.new_deaths[r_i, :]

plot_area_cases_curve(expected_cases, psi_cases, new_cases, data.Ds)
plt.ylim([1, 10000])
plt.ylabel("Cases", fontsize=10, color="tab:blue")

plt.minorticks_off()
plt.twinx()

plot_area_deaths_curve(expected_deaths, psi_deaths, new_deaths, data.Ds)
plt.ylabel("Deaths", fontsize=10, rotation=-90, color="tab:red", va='bottom')
plt.ylim([1, 10000])
plt.yticks([])
plt.minorticks_off()


plt.tight_layout()
plt.savefig('figures/appendix/FigFits.pdf', bbox_inches='tight')

## Posterior NPI Effect Correlation

In [ ]:
corr_dict = {
    'Night clubs closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
        'main': False,
    },
    'Leisure and entertainment\nvenues closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Gastronomy closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
         'main': False,
    },
    'Retail and close-contact\nservices closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
        'main': False,
    },
    'Public gatherings limited to 1 person': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
#                  'Public Indoor Gathering Person Limit - 2',
#                  'Public Indoor Gathering Person Limit - 10',
#                  'Public Indoor Gathering Person Limit - 30',
#                  'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
#                  'Public Indoor Gathering Person Limit - 10',
#                  'Public Indoor Gathering Person Limit - 30',
#                  'Extra Public Indoor Household Limit',
                ],
        "type": "exclude",
        'color': cols[4],
    },
    'Public gatherings limited to ≤10 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 10',
#                  'Public Indoor Gathering Person Limit - 30',
#                  'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Public Indoor Gathering Person Limit - 30',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    'Public gatherings limited to 2 households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 'Extra Public Indoor Household Limit',
                ],
        'type': 'exclude',
        'color': cols[4],
    },
    
    'All household mixing in private banned': {
        'npis': ['Private Indoor Gathering Person Limit - 1',
#                  'Private Indoor Gathering Person Limit - 2',
#                  'Private Indoor Gathering Person Limit - 10',
#                  'Private Indoor Gathering Person Limit - 30',
#                  'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
#                  'Private Indoor Gathering Person Limit - 10',
#                  'Private Indoor Gathering Person Limit - 30',
#                  'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤10 people': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 10',
#                  'Private Indoor Gathering Person Limit - 30',
#                  'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to ≤30 people': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 10',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 10',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'Household mixing in private\nlimited to 2 households': {
        'npis': [#'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
#                  'Private Indoor Gathering Person Limit - 10',
#                  'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[5],
    },
    'All education institutions closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
        'main': False,
    },
    'Night time curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
    'Stricter mask-wearing\npolicy': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[1],
        'main': True
    },
}

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

comb_cm_effects, new_names = combine_npi_samples(corr_dict, alpha_i, cm_names)

cormat = np.corrcoef(comb_cm_effects.T)

plt.figure(figsize=(7.5, 6), dpi=300)
sns.heatmap(cormat, vmin=-1, vmax=1, cmap="icefire")
plt.yticks(0.5+np.arange(len(new_names)), new_names, rotation=0, fontsize=6);
plt.xticks(0.5+np.arange(len(new_names)), new_names, rotation=45, ha='left', fontsize=6);
plt.gca().xaxis.tick_top()
plt.tight_layout()
plt.savefig('figures/appendix/FigCorr.pdf', bbox_inches='tight')

In [ ]:
cormat[cormat>0.999] = 0

In [ ]:
i, j = np.nonzero(np.abs(cormat) > 0.4)

In [ ]:
npi_names = list(corr_dict.keys())

In [ ]:
for a, b in zip(i, j):
    print(f"{npi_names[a]} and {npi_names[b]} {cormat[a, b]:.2f}")

## Random Walk Noise Scale Posterior

In [ ]:
x = np.linspace(1e-5, 0.3, 100000)
prior_density = 2/(0.15*np.sqrt(np.pi*2))*np.exp(-0.5 * (x/0.15)**2)

In [ ]:
plt.figure(figsize=(3, 3), dpi=300)
plt.plot(x, prior_density, label='prior')
sns.kdeplot(full_trace.posterior.r_walk_noise_scale.data.flatten(), label='posterior')
plt.xlabel("Random Walk Noise Scale", fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.ylabel("density", fontsize=10)
plt.legend(fontsize=8)

plt.savefig('figures/appendix/FigRandomWalkNoiseScale.pdf', bbox_inches='tight')